In [1]:
! pip install openai
import dataclasses
import logging
import math
import ast
import re
import os
import io
import sys
import time
import json
import tqdm
import copy
import openai
import pandas as pd

from typing import Optional, Sequence, Union
from cleantext import clean
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

# from openai import openai_object
openai.api_key = ""

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


## Load data 

In [2]:
# current_directory = os.getcwd()
qaa = pd.read_csv("raw_input/qd_immigration.csv", encoding = "utf8") # LLMs usually requires input text data with character encoding utf8
qaa.head()

,Unnamed: 0,ID,Question,Answer,Vote Status,Timestamp
0,0,1,H-1B to EB-2 process . I would like to know If...,The employer can start the green card process ...,up,NaN
1,1,2,Eligibility for and the process of EB-3 to EB-...,You can always go up and you can always go dow...,up,NaN
2,2,3,How can I qualify for EB-1C/International Mana...,1. Please discuss your job description that is...,up,NaN
3,3,4,Downgrading from EB-2 to EB-3 . My wife and I ...,I see no problem applying for EB-3 and then us...,up,NaN
4,4,5,"How to get H-1B approved for three years, not ...",1. Typically the only way you can get three ye...,up,NaN


#### Check the number of missing values

In [3]:
print(qaa.isna().sum())

Unnamed: 0      0
ID              0
Question        0
Answer          0
Vote Status     0
Timestamp      96
dtype: int64


## Data preprocessing

#### Stop words removal and stemming (Optional)

In [115]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [116]:
stop = stopwords.words('english')
# These words are important for the problem. Exclude them from the stop words.
excluding = ['against', 'not', 'don', "don't",'ain', 'aren', "aren't", 'couldn', "couldn't",
             'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 
             'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't",
             'needn', "needn't",'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', 
             "weren't", 'won', "won't", 'wouldn', "wouldn't"]
# New stop word list
stop_words = [word for word in stop if word not in excluding]

snow = SnowballStemmer('english')

def process_text(texts): 
    final_text_list=[]
    for sent in texts:
        
        # Check if the sentence is a missing value
        if isinstance(sent, str) == False:
            sent = ""
            
        filtered_sentence=[]
        
        sent = sent.lower() # Lowercase 
        sent = sent.strip() # Remove leading/trailing whitespace
        sent = re.sub('\s+', ' ', sent) # Remove extra space and tabs
        sent = re.compile('<.*?>').sub('', sent) # Remove HTML tags/markups:

        for w in word_tokenize(sent):
            # We are applying some custom filtering here, feel free to try different things
            # Check if it is not numeric and its length>2 and not in stop words
            if(not w.isnumeric()) and (len(w)>2) and (w not in stop_words):  
                # Stem and add to filtered list
                filtered_sentence.append(snow.stem(w))
        final_string = " ".join(filtered_sentence) #final string of cleaned words
 
        final_text_list.append(final_string)
        
    return final_text_list

stop_stem=False
if stop_stem:
    question_l_raw, answer_l_raw = qaa["Question"].to_list(), qaa["Answer"].to_list()
    qaa["Question"] = process_text(question_l_raw)
    qaa["Answer"] = process_text(answer_l_raw)

#### Personal Identification Information (PII) removal and other preprocessing using cleantext

In [117]:
! pip install clean-text
f_clean = lambda qaa_pair_raw : clean(qaa_pair_raw,
    fix_unicode=True,               # fix various unicode errors
    to_ascii=True,                  # transliterate to closest ASCII representation
    lower=True,                     # lowercase text
    no_line_breaks=False,           # fully strip line breaks as opposed to only normalizing them
    no_urls=False,                  # replace all URLs with a special token
    no_emails=False,                # replace all email addresses with a special token
    no_phone_numbers=False,         # replace all phone numbers with a special token
    no_numbers=False,               # replace all numbers with a special token
    no_digits=False,                # replace all digits with a special token
    no_currency_symbols=False,      # replace all currency symbols with a special token
    no_punct=False,                 # remove punctuations
    replace_with_punct="",          # instead of removing punctuations you may replace them
    replace_with_url="<URL>",
    replace_with_email="<EMAIL>",
    replace_with_phone_number="<PHONE>",
    replace_with_number="<NUMBER>",
    replace_with_digit="0",
    replace_with_currency_symbol="<CUR>",
    lang="en"                       # set to 'de' for German special handling
)
#import pdb; pdb.set_trace()
question_l_raw = qaa["Question"].to_list()
question_l = [f_clean(p) for p in question_l_raw]

answer_l_raw = qaa["Answer"].to_list()
answer_l =  [f_clean(p) for p in answer_l_raw]


qaa["Question"] = question_l
qaa["Answer"] = answer_l


#### Dataset customized clenaup

In [118]:
answer_l_raw = qaa["Answer"].to_list()
qaa["Answer"] = [re.compile(r'<.*?>|More\.\.\.', flags=re.IGNORECASE).sub('', p) for p in answer_l_raw] # Remove HTML tags/markups: 

#### Data structure conversion

In [119]:
qaa_list = qaa[["Question", "Answer"]].to_dict('records')
print(len(qaa_list))
print(qaa_list[:10])

96
[{'Question': 'h-1b to eb-2 process . i would like to know if i have an advanced degree (masters engineering management) and my employer filed my h-1b and if the lottery is picked can i initiate the green card process? i also heard that there is a minimum salary cap for eb-2 advanced degree.', 'Answer': 'the employer can start the green card process at any time, even before you join. to see the salary figures by county and profession, you can review this link -https://www.flcdatacenter.com/'}, {'Question': 'eligibility for and the process of eb-3 to eb-2 porting . can you talk about this upgrade process from eb3 to eb2 for pending i-485? does it need another i-485 application or just a letter to uscis?', 'Answer': "you can always go up and you can always go down as long as your perm was filed as an eb-2. if you filed your prm as an eb-3 then you're not going to be able to upgrade to eb-2. but if your perm was filed as an eb-2 you can downgrade i-140 eb-3. you can go back upgrade toe

### Prompt Engineering

In [120]:
prompts= """Paraphrase the below question and answer pair in 3 different ways.
Try not to repeat the verb for each pair to maximize diversity.
Return everything in an array of JSON object in this format: ######{"_question":"string", "_answer":"string"}
Seperate each pair with "######" rather than commas.
"""
print(len(prompts))

294


In [ ]:
def encode_prompt_QA(prompts=prompts, QA_list=[]):
    """Encode multiple prompt instructions into a single string."""
    listof_prompt_QA = []
    for idx, task_dict in enumerate(QA_list):
        single_prompt_QA = prompts+"\n"
        (question, answer) = task_dict["Question"], task_dict["Answer"]
        question = re.sub(r"\s+", " ", question).strip().rstrip(":")
        answer = "<noinput>" if answer.lower() == "" else answer
        single_prompt_QA += f"######\n"
        single_prompt_QA += "{"
        single_prompt_QA += f"\"_question\": \"{question}\",\n"
        single_prompt_QA += f"\"_answer\": \"{answer}\""
        single_prompt_QA += "}"
        # single_prompt_QA += f"###\n"
        listof_prompt_QA.append(single_prompt_QA)
    return listof_prompt_QA

qaa_list_encoded = encode_prompt_QA(prompts, qaa_list)
print("len(qaa_list_encoded): ", len(qaa_list_encoded))
for id, qa in enumerate(qaa_list_encoded[:10]):
    print(qa)
    print("\n")

#### Generate synthetic data with low temparature

In [122]:
qaa_augmented_raw = []

In [123]:
for id, batch_inputs_string in enumerate(qaa_list_encoded):
    print(id, "\n") # "\n\n\n", "input:\n", batch_inputs_string, "\n"
    completion_batch = openai.Completion.create(
        prompt=batch_inputs_string, 
        model="text-davinci-003",
        temperature = 0.2,
        max_tokens = 1000 # The maximum number of tokens to generate in the completion
        )
    results_string = completion_batch['choices'][0]['text']
    print(results_string, "\n\n\n")
    qaa_augmented_raw.append(results_string)

0 



######
{"_question": "Can I initiate the green card process if my employer files an H-1B and it is picked in the lottery?",
"_answer": "Yes, the employer can begin the green card process at any time, even before you start working. To check the salary requirements by county and profession, you can look at this link -https://www.flcdatacenter.com/"}

######
{"_question": "If my employer files an H-1B and it is selected in the lottery, can I begin the green card process?",
"_answer": "Yes, the employer can initiate the green card process before you even start working. To view the salary limits by county and job, you can refer to this link -https://www.flcdatacenter.com/"}

######
{"_question": "If my employer files an H-1B and it is chosen in the lottery, am I able to start the green card process?",
"_answer": "Yes, the employer can initiate the green card process before you begin working. To check the salary caps by county and occupation, you can look at this link -https://www.flcd

#### Generate synthetic data with high temperature

In [124]:
qaa_augmented_raw = []
for id, batch_inputs_string in enumerate(qaa_list_encoded):
    print(id, "\n") # "\n\n\n", "input:\n", batch_inputs_string, "\n"
    completion_batch = openai.Completion.create(
        prompt=batch_inputs_string, 
        model="text-davinci-003",
        temperature = 0.5,
        max_tokens = 2000 # The maximum number of tokens to generate in the completion
        )
    results_string = completion_batch['choices'][0]['text']
    print(results_string, "\n\n\n")
    qaa_augmented_raw.append(results_string)

0 



######
{"_question": "Do I need to wait before beginning the green card process if my employer files an H-1B and the lottery is picked?",
"_answer": "No, the employer can initiate the green card process prior to your joining. You can review the salary figures by county and profession at this link -https://www.flcdatacenter.com/"}

######
{"_question": "Can I start the green card process if my employer has filed an H-1B and it has been selected in the lottery?",
"_answer": "Yes, the employer can begin the green card process even before you join. Check out this link -https://www.flcdatacenter.com/ to view the salary figures by county and profession."}

######
{"_question": "If my employer files an H-1B and it is selected in the lottery, when can I start the green card process?",
"_answer": "The employer can initiate the green card process prior to your joining. You can examine the salary figures by county and profession at this link -https://www.flcdatacenter.com/"} 



1 



#####

### dump raw output to a file


In [125]:
def _make_w_io_base(f, mode: str):
    if not isinstance(f, io.IOBase):
        f_dirname = os.path.dirname(f)
        if f_dirname != "":
            os.makedirs(f_dirname, exist_ok=True)
        f = open(f, mode=mode)
    return f

def jdump(obj, f, mode="w", indent=4, default=str):
    """Dump a str or dictionary to a file in json format.

    Args:
        obj: An object to be written.
        f: A string path to the location on disk.
        mode: Mode for opening the file.
        indent: Indent for storing json dictionaries.
        default: A function to handle non-serializable entries; defaults to `str`.
    """
    f = _make_w_io_base(f, mode)
    if isinstance(obj, (dict, list)):
        json.dump(obj, f, indent=indent, default=default)
    elif isinstance(obj, str):
        f.write(obj)
    else:
        raise ValueError(f"Unexpected type: {type(obj)}")
    f.close()

In [126]:
output_name = "output_raw_96.json"
output_dir="./raw_input"
jdump(qaa_augmented_raw, os.path.join(output_dir, output_name))

### Read json file

In [127]:
def _make_r_io_base(f, mode: str):
    if not isinstance(f, io.IOBase):
        f = open(f, mode=mode)
    return f

def jload(f, mode="r"):
    """Load a .json file into a dictionary."""
    f = _make_r_io_base(f, mode)
    jdict = json.load(f)
    f.close()
    return jdict

output_raw = jload("raw_input/output_raw_96.json")
#import pdb; pdb.set_trace()
output_raw[0]

'\n\n######\n{"_question": "Do I need to wait before beginning the green card process if my employer files an H-1B and the lottery is picked?",\n"_answer": "No, the employer can initiate the green card process prior to your joining. You can review the salary figures by county and profession at this link -https://www.flcdatacenter.com/"}\n\n######\n{"_question": "Can I start the green card process if my employer has filed an H-1B and it has been selected in the lottery?",\n"_answer": "Yes, the employer can begin the green card process even before you join. Check out this link -https://www.flcdatacenter.com/ to view the salary figures by county and profession."}\n\n######\n{"_question": "If my employer files an H-1B and it is selected in the lottery, when can I start the green card process?",\n"_answer": "The employer can initiate the green card process prior to your joining. You can examine the salary figures by county and profession at this link -https://www.flcdatacenter.com/"}'

### Postprocess the Raw Outputs

In [128]:
QApair_dict = []
error_list = []
for raw in qaa_augmented_raw:
    pairs = raw.split("######")
    for id, item in enumerate(pairs):
        # print(id)
        if not item.isspace():
            try:
                str2dict = ast.literal_eval(item)
                QApair_dict.append(str2dict)
            except Exception as e: 
                print(e)
                print(item)
                print("\n\n\n")
                error_list.append(item)
                # print(type(str2dict))
                # print(str2dict['_question'])
                # print(str2dict['_answer'])
print(len(QApair_dict))
# QApair_dict

276


In [129]:
QApair_df = pd.DataFrame(QApair_dict)
QApair_df

,_question,_answer
0,Do I need to wait before beginning the green c...,"No, the employer can initiate the green card p..."
1,Can I start the green card process if my emplo...,"Yes, the employer can begin the green card pro..."
2,If my employer files an H-1B and it is selecte...,The employer can initiate the green card proce...
3,What is the process for upgrading from EB-3 to...,It is possible to upgrade from EB-3 to EB-2 as...
4,What is involved in the EB-3 to EB-2 porting p...,"If the PERM was submitted as an EB-2, you can ..."
...,...,...
271,Does my wife qualify for EB2 processing as an ...,OT's do not have a special category like physi...
272,What is the process for my wife to apply for E...,OT's do not have a special pathway like physic...
273,Can my wife use her occupational therapist deg...,OT's do not have a dedicated category like phy...
274,I have 3 years of experience in the IT field a...,I understand you are inquiring if you can appl...


In [130]:
QApair_df.to_csv("output/output_qa_augment.csv", index=False)